In [1]:
# add path (for local)
import sys
repo_root = "../"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [2]:
from typing import Optional
from guacamol.goal_directed_generator import GoalDirectedGenerator
from guacamol.scoring_function import ScoringFunction
from guacamol.assess_goal_directed_generation import assess_goal_directed_generation
from rdkit import Chem

from reward import MolReward
from utils import conf_from_yaml, generator_from_conf

yaml_path = "config/mcts_guacamol.yaml"

class GuacaMolReward(MolReward):
    def __init__(self, scoring_function: ScoringFunction):    
        self.scoring_function = scoring_function
        
    # implement
    def mol_objective_functions(self):
        def raw_score(mol):
            smiles = Chem.MolToSmiles(mol)
            return self.scoring_function.score(smiles)

        return [raw_score]

    # implement
    def reward_from_objective_values(self, objective_values):
        score = objective_values[0]
        return score

class V3DeNovoGenerator(GoalDirectedGenerator):
    # implement
    def generate_optimized_molecules(self, scoring_function: ScoringFunction, number_molecules: int, starting_population: Optional[list[str]] = None) -> list[str]:        
        conf = conf_from_yaml(yaml_path, repo_root)
        generator = generator_from_conf(conf)
        generator.reward = GuacaMolReward(scoring_function=scoring_function)
        generator.generate(max_generations=number_molecules, time_limit=conf.get("time_limit"))
        
        return generator.generated_keys(last=number_molecules)

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
/opt/anaconda3/envs/v3-forge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
assess_goal_directed_generation(V3DeNovoGenerator())

seed: 1752564374
Starting generation...
<Best reward updated> 1 - time: 0.04, reward: 0.2474, node: CCCN[C@@]1(C#N)CC[C@H](Sc2nnnn2CC)C1
<Best reward updated> 3 - time: 0.11, reward: 0.2943, node: Fc1ccc(O[C@H]2C[C@H](Cc3cnccn3)C2)cc1F
<Best reward updated> 17 - time: 0.74, reward: 0.3784, node: O=C([O-])[C@@H]1OCC[C@H]1C[NH2+]Cc1cccs1
<Best reward updated> 56 - time: 2.35, reward: 0.3784, node: c1cnc(N2CCC[C@H](OCc3nccs3)C2)nc1
<Best reward updated> 63 - time: 2.58, reward: 0.5355, node: O=C(OC(F)(F)F)N1C[C@H](CO)C[C@@H]1OCc1cccs1
100 - time: 4.78, average over 100: 0.1547
Generation finished.
seed: 1752564384
Starting generation...
<Best reward updated> 1 - time: 0.00, reward: 0.3213, node: CSc1ccc2cc(Cl)c(S[C@@H](C)C(=O)N3CCOCC3)nc2n1
<Best reward updated> 4 - time: 0.09, reward: 0.3236, node: O[C@H]1CCCN(c2ccncc2)C1
<Best reward updated> 5 - time: 0.12, reward: 0.4426, node: O=S(=O)(Nc1cc(F)nc2ccccc12)c1ccc(Cl)nc1
<Best reward updated> 14 - time: 0.64, reward: 0.4719, node: FC(F)Oc